# AI Agents with Generative AI Hub

Everyone is talking about AI Agents and how they are going to make LLMs more powerful. AI Agents are basically systems that have a set of tools that they can use and the LLM is the agents brain to decide which tool to use when. Some systems are more agentic than others, depending on the amount of autonomy and decision-making power they possess. 

If you want to know more about AI Agents and how to build one from scratch, check out this [Devtoberfest session](https://community.sap.com/t5/devtoberfest/getting-started-with-agents-using-sap-generative-ai-hub/ev-p/13865119).

👉 Before you start you will need to add the constant `LLM_DEPLOYMENT_ID` to [variables.py](variables.py). You can use the `gpt-4.1-mini` model again that we deployed earlier. You find the deployment ID in SAP AI Launchpad.
👉 You will need to run `pip install langgraph` to install the necessary [LangGraph Python package](https://langchain-ai.github.io/langgraph/agents/agents/).

In [1]:
import init_env
import variables

init_env.set_environment_variables()

### Import the packages you want to use

In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

from langsmith import Client  # ✅ Correct import
from langgraph.prebuilt import create_react_agent

from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
from gen_ai_hub.proxy.langchain.openai import OpenAIEmbeddings


We will give the agent different tools to use. The first tool will be [access to Wikipedia](https://python.langchain.com/docs/integrations/tools/wikipedia/). This way instead of answering from it's training data, the model will check on Wikipedia articles first.

In [3]:
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# Let's check if it works
print(wikipedia.run("Tbilisi"))

# Assign wikipedia to the set of tools
tools = [wikipedia]

Page: Tbilisi
Summary: Tbilisi (  tə-bil-EE-see, tə-BIL-iss-ee; Georgian: თბილისი, pronounced [ˈtʰbilisi] , Georgian: ტფილისი, romanized: t'pilisi [tʼpʰilisi]) is the capital and largest city of Georgia, located on the banks of the Kura River. With more than 1.3 million inhabitants, it contains almost one third of the country's population. Tbilisi was founded in the fifth century AD by Vakhtang I of Iberia and has since served as the capital of various Georgian kingdoms and republics. Between 1801 and 1917, then part of the Russian Empire, it was the seat of the Caucasus Viceroyalty, governing both the northern and the southern sides of the Caucasus.
Because of its location at the crossroads between Europe and Asia, and its proximity to the lucrative Silk Road, throughout history, Tbilisi has been a point of contention among various global powers. To this day, the city's location ensures its position as an important transit route for energy and trade projects. Tbilisi's history is refl

In [5]:
# For agents the initial prompt including all the instructions is very important. 
# LangChain already has a good set of prompts that we can reuse here.
hub = Client()  # ✅ Create langsmith client
prompt = hub.pull_prompt("hwchase17/react")
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [8]:
import importlib
variables = importlib.reload(variables)

# Create a chat model instance which will act as the brain of the agent
llm = ChatOpenAI(deployment_id=variables.LLM_DEPLOYMENT_ID)

# Create an agent with the llm, tools and prompt
agent = create_react_agent(llm, tools)

# Let's ask try the agent and ask about the city
result = agent.invoke({
            "messages": [
                ("user", f"{prompt}\n\nTell me about Tbilisi!")
            ]
        })

/var/folders/hb/9f88t7n14654_h2tgsf1y_hh0000gn/T/ipykernel_20131/3039866074.py:8: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(llm, tools)


## Printing the result of the agent

In [9]:
# Printing the results
from IPython.display import Markdown, display

# Display as formatted markdown
final_response = result['messages'][-1].content
display(Markdown(f"## Agent Response\n\n{final_response}"))

# Or show the reasoning process
print("=== Agent Thinking Process ===")
for msg in result['messages']:
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        print(f"🔧 Agent decided to use tool: {msg.tool_calls[0]['name']}")
    elif msg.__class__.__name__ == 'ToolMessage':
        print(f"📋 Tool returned data (length: {len(msg.content)} chars)")
    elif msg.__class__.__name__ == 'AIMessage' and msg.content:
        print(f"🤖 Final response ready")

## Agent Response

Question: Tell me about Tbilisi!
Thought: I should provide comprehensive information about Tbilisi including its location, historical significance, cultural importance, notable landmarks, and climate.
Action: wikipedia
Action Input: Tbilisi
Observation: Tbilisi is the capital and largest city of Georgia, located on the banks of the Kura River. It has more than 1.3 million inhabitants, comprising almost one third of the country's population. Founded in the fifth century AD by Vakhtang I of Iberia, it has served as the capital of various Georgian kingdoms and republics. Between 1801 and 1917, as part of the Russian Empire, it was the seat of the Caucasus Viceroyalty.

Tbilisi's location at the crossroads between Europe and Asia, near the Silk Road, made it a point of contention among various global powers. The city's strategic position remains important for energy and trade projects. 

Architecturally, Tbilisi features a mix of medieval, neoclassical, Beaux Arts, Art Nouveau, Stalinist, and Modern styles. It historically hosts people of diverse cultural, ethnic, and religious backgrounds, predominantly Eastern Orthodox Christian.

Notable tourist destinations include cathedrals Sameba and Sioni, Freedom Square, Rustaveli Avenue, Aghmashenebeli Avenue, medieval Narikala Fortress, the pseudo-Moorish Opera Theater, and the Georgian National Museum. The climate ranges from 20 to 32 °C in summer and −1 to 7 °C in winter.

Thought: I now know the final answer
Final Answer: Tbilisi is the capital and largest city of Georgia, located on the banks of the Kura River with over 1.3 million inhabitants. Founded in the fifth century AD, it has historically been a key city at the crossroads of Europe and Asia, near the Silk Road, giving it strategic importance for energy and trade. The city's architecture is a blend of medieval to modern styles. It is home to a diverse cultural and religious population, primarily Eastern Orthodox Christians. Key landmarks include the Sameba and Sioni cathedrals, Freedom Square, Rustaveli Avenue, Narikala Fortress, and the Georgian National Museum. The climate in Tbilisi ranges from warm summers (20-32 °C) to cool winters (-1 to 7 °C).

=== Agent Thinking Process ===
🔧 Agent decided to use tool: wikipedia
📋 Tool returned data (length: 4000 chars)
🤖 Final response ready


## Give the agent access to our HANA vector store
Now the agent can use Wikipedia but wouldn't it be nice if the agent could pick between resources and decide when to use what?

In [12]:
# LangChain & HANA Vector Engine - legacy integration: https://python.langchain.com/api_reference/community/vectorstores/langchain_community.vectorstores.hanavector.HanaDB.html
#from langchain_community.vectorstores.hanavector import HanaDB

# SAP HANA integration with LangChain: https://pypi.org/project/langchain-hana 
from langchain_hana import HanaDB
from langchain_core.tools import create_retriever_tool

# connect to HANA instance
connection = init_env.connect_to_hana_db()
connection.isconnected()

# Reference which embedding model, DB connection and table to use
embeddings = OpenAIEmbeddings(deployment_id=variables.EMBEDDING_DEPLOYMENT_ID)
db = HanaDB(
    embedding=embeddings, connection=connection, table_name=variables.EMBEDDING_TABLE
)

# Create a retriever instance of the vector store
retriever = db.as_retriever(search_kwargs={"k": 2})

# We need to add a description to the retriever so that the llm knows when to use this tool.
retriever_tool = create_retriever_tool(
    retriever,
    "SAP-orchestration-service-docu",
    "Search for information SAP's orchestration service. For all inquiries specifically related to the Generative AI Hub and SAP's orchestration service, this tool must be used without exception!",
)

## Ask the Agent

You can ask the agent anything in general and it will try to find an entry from Wikipedia, unless you ask about SAP's orchestration service. Then it will get the information from SAP HANA vector store, where it holds the documentation, instead of Wikipedia.

In [14]:
# Now let's add the retriever as a tool
tools = [wikipedia, retriever_tool]

# And create the agent again with the two tools, wikipedia and the HANA vector store (retriever)
# Create an agent with the llm, tools and prompt
agent = create_react_agent(llm, tools)

# First ask: What is Data Masking?
# Then ask: What is Data Masking in SAP's Generative AI Hub?
# Pay attention to the response! Do you see what is happening now?
result = agent.invoke({
            "messages": [
                ("user", f"{prompt}\n\nWhat is Data Masking in SAP's Generative AI Hub?")
            ]
        })

/var/folders/hb/9f88t7n14654_h2tgsf1y_hh0000gn/T/ipykernel_20131/686678562.py:6: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(llm, tools)


In [15]:
# Display as formatted markdown
final_response = result['messages'][-1].content
display(Markdown(f"## Agent Response\n\n{final_response}"))

# Or show the reasoning process
print("=== Agent Thinking Process ===")
for msg in result['messages']:
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        print(f"🔧 Agent decided to use tool: {msg.tool_calls[0]['name']}")
    elif msg.__class__.__name__ == 'ToolMessage':
        print(f"📋 Tool returned data (length: {len(msg.content)} chars)")
    elif msg.__class__.__name__ == 'AIMessage' and msg.content:
        print(f"🤖 Final response ready")

## Agent Response

Question: What is Data Masking in SAP's Generative AI Hub?
Thought: I found that Data Masking in SAP's Generative AI Hub involves a process with providers using SAPDataPrivacyIntegration, specifically employing the method of anonymization. This method targets entities such as SAP internal IDs and includes masking the grounding input to protect sensitive data.
Final Answer: Data Masking in SAP's Generative AI Hub is a process that uses anonymization methods to protect sensitive data, particularly SAP internal IDs, by masking the grounding input. This ensures privacy and data protection when using AI services.

=== Agent Thinking Process ===
🔧 Agent decided to use tool: SAP-orchestration-service-docu
📋 Tool returned data (length: 503 chars)
🤖 Final response ready
